In [17]:
import speech_recognition as speech_recognize
import asyncio
import nest_asyncio
nest_asyncio.apply()




recognizer_threshold = 800
speak_language = 'Russia'

bot_name = 'Аура'
cycle_stop_phrase = 'тормози'


restricted_parts = ['speaker', 'outp', 'динамик']
required_syms = '()'

recognizer = speech_recognize.Recognizer()
recognizer.energy_threshold = recognizer_threshold

loop = asyncio.get_event_loop()



In [2]:
# Не доделано, поскольку по-умолчанию библиотека слушает дефолтный микрофон системы
async def select_input_device():
    unique_micro_names = []
    for name in speech_recognize.Microphone.list_microphone_names():
        if name:
            name = name.encode('cp1251').decode('utf8')
            if name not in unique_micro_names and sum([1 if required in name else 0 for required in required_syms]) == 2 and not (sum([1 if restricted in name.lower() else 0 for restricted in restricted_parts])):
                unique_micro_names.append(name)
    
    print('Выберите одно из представленных устройств:\n')

    input_text = input(''.join([f"\n** {unique_micro_names.index(name)} ** {name}" for name in unique_micro_names]))

    try:
        input_text = int(input_text)
        return()

    except:
        print('Микрофона с таким номером не существует.')
        return(False)
    # print(f'\n** '.join(unique_micro_names))

In [20]:
class Bot():
    def __init__(self, recognizer : speech_recognize.Recognizer):
        self.recognizer = recognizer


    async def recognize_speech(self, recognize_timeot = 1):
        with speech_recognize.Microphone() as source:
            # print("Скажите что-нибудь:")

            try:
                audio = self.recognizer.listen(source, timeout = recognize_timeot)

            except speech_recognize.WaitTimeoutError:
                out = ''
                # print('Тишина')
                return(out)


        try:
            text = await asyncio.to_thread(self.recognizer.recognize_google, audio, language="ru-RU")
            out = str(text)
            # print("Вы сказали:", text)

        except speech_recognize.UnknownValueError:
            out = ''
            # print("Распознать речь не удалось")
        
        return(out)


    async def commands_redirecter(self, text):
        # 1 элемент вывода - контент, 2 - надо ли остановить цикл
        
        text = text.lower()

        if bot_name.lower() in text:
            print(f'{bot_name} был вызван')
            return('Внемлю', False)
        
        elif cycle_stop_phrase.lower() in text:
            print('Цикл остановлен')
            return('Цикл остановлен', True)
        
        else:
            return(text, False)



    async def main(self):
        is_stop = False
        while not is_stop and isinstance(is_stop, bool):
            recognized = await self.recognize_speech(0.4)

            data = await self.commands_redirecter(recognized)
            content = data[0]
            is_stop = data[1]

            if len(content):
                print(content)



In [35]:
import pyttsx3

def synthesize_with_voice(text, voice_name, lang="en"):
    engine = pyttsx3.init()
    voices = engine.getProperty('voices')
    engine.setProperty('voice', voices[1].id)
    engine.setProperty('rate', 220)  # Устанавливаем скорость озвучивания

    engine.say(text)
    engine.runAndWait()

text = "Привет, это синтезированный голосовой текст."
voice_name = "russian"  # Название русского голоса на вашей системе

synthesize_with_voice(text, voice_name, lang="ru")


In [31]:
engine = pyttsx3.init()


'HKEY_LOCAL_MACHINE\\SOFTWARE\\Microsoft\\Speech\\Voices\\Tokens\\TTS_MS_RU-RU_IRINA_11.0'

In [26]:
from gtts import gTTS
from pydub import AudioSegment
from pydub.playback import play
import io

def synthesize_and_play_audio(text, lang="en"):
    tts = gTTS(text, lang=lang, )
    
    # Создаем объект BytesIO для работы с данными в памяти
    mp3_data = io.BytesIO()
    tts.write_to_fp(mp3_data)
    
    # Перематываем в начало данных для их чтения
    mp3_data.seek(0)
    
    audio = AudioSegment.from_mp3(mp3_data)
    play(audio)

text = "Хотелось бы мне говорить побыстрее."
synthesize_and_play_audio(text, lang="ru")



In [21]:
bot_object = Bot(recognizer)

In [36]:
loop.run_until_complete(bot_object.main())

алло
алло
армавир
Цикл остановлен
Цикл остановлен


NameError: name 'main' is not defined

---